In [2]:
colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    !git clone https://github.com/deepmind/pycolab.git
    !git clone https://github.com/nicoladainese96/RelationalModule.git
    !pip install pycolab

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from RelationalModule import ActorCritic
from RelationalModule import train_agent as train
from importlib import reload

In [4]:
reload(train)

<module 'RelationalModule.train_agent' from '/home/nicola/Nicola_unipd/MasterThesis/RelationalDeepRL/RelationalModule/train_agent.py'>

In [5]:
game_params = dict(grid_size=7,
                solution_length=[0],
                num_forward = [0], # number of distractors
                num_backward=[0], # just set to 0 for now
                branch_length=1 # length of forward distractors
               )

In [6]:

HPs = dict(action_space=4,
           lr=5e-3,
           gamma=0.99,
           TD=True,
           twin=True,
           tau=0.2,
           n_steps=50
           )
if colab:
    HPs['device'] = 'cuda'
else:
    HPs['device'] = 'cpu'

print('device: ', HPs['device'])  
agent = ActorCritic.BoxWorldA2C(**HPs)

device:  cpu


In [7]:
%%time
results = train.train_boxworld(agent, game_params, n_episodes = 5000, return_agent=True)
score, asymptotic_score, asymptotic_std, trained_agent, time_profile = results

/home/nicola/anaconda3/envs/torch/lib/python3.7/site-packages/pycolab/ascii_art.py:318: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  art = np.vstack(np.fromstring(line, dtype=np.uint8) for line in art)


Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Episode 1 - reward: 10
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Keys:  dict_keys(['#', ' ', '.', '*'])
Ke

KeyboardInterrupt: 

On my PC:

 Average time for playing an episode: 1.97428691 <br>
 Average time for updating the agent: 26.46678982

 On Colab notebook:

 Average time for playing an episode:  1.11658633 <br>
 Average time for updating the agent: 18.63836179


With GPU

Average time for playing an episode: 0.43020332 <br>
Average time for updating the agent: 0.64481306


In [ ]:
plt.figure(figsize=(8,6))
n_epochs = np.arange(100, len(score))
average_score = np.array([np.mean(score[i:i+100]) for i in range(len(score)-100)])
plt.plot(n_epochs, average_score, alpha=0.9)
plt.title("Performance", fontsize=16)
plt.xlabel("Number of epochs", fontsize=16)
plt.ylabel("Total reward (max=500)", fontsize=16)
plt.show()

In [ ]:
if colab:
    %cd ~
    parent_dir = "/content/gdrive/My Drive/Colab Notebooks/"
    save_dir  = "RelationalTrained/"
    %cd "{parent_dir}"
    !mkdir "{save_dir}"
    torch.save(trained_agent, save_dir+"trained_agent")

In [ ]:
reload = False
if reload:
    %cd ~
    parent_dir = "/content/gdrive/My Drive/Colab Notebooks/"
    save_dir  = "RelationalTrained/"
    reloaded_agent = torch.load(parent_dir+save_dir+"trained_agent")